In [330]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [350]:
class TransducerLoss(torch.nn.Module):
    def __init__(self):
        super(TransducerLoss, self).__init__()
        
    def show_alignment(self, log_alpha):
        plt.imshow(log_alpha.cpu().data.numpy()); plt.show()
    
    def compute_log_alpha(self, net_out, y, blank):
        net_out = torch.nn.functional.log_softmax(net_out, dim=1) # not needed if net_out already logsoftmax, but warp-ctc does this
        T = len(net_out)
        U = len(y)
        S = 2*U + 1
        y_prime = []
        for i in range(S):
            label = blank if (i+1) % 2 else y[int(i/2)].item()
            y_prime.append(label)
            
        log_alphas = []
        for t in range(T):
            log_alpha_t = torch.zeros(U)
            for u in range(U):
                
                
            log_alphas.append(log_alpha_t)
            
        log_alpha = torch.stack(log_alphas)
        return log_alpha
    
    def forward(self,encoder_out,decoder_out,targets,input_lengths,target_lengths,reduction="none",blank=0):
        """
        encoder_out: FloatTensor (max(input_lengths), N, #labels)
        decoder_out: FloatTensor (max(target_lengths), N, #labels)
        targets: LongTensor (N, max(target_lengths))
        input_lengths: LongTensor (N)
        target_lengths: LongTensor (N)
        reduction: "none", "avg"
        blank: int
        """
        batch_size = len(input_lengths)
        losses = []
        for i in range(0, batch_size):
            net_out = log_probs[:input_lengths[i], i, :]
            y = targets[i, :target_lengths[i]]
            log_alpha = self.compute_log_alpha(net_out, y, blank) # shape (T, 2U + 1)
            loss = -torch.logsumexp(log_alpha[-1, -2:], dim=0) 
            losses.append(loss)
        losses = torch.stack(losses)
        if reduction=="none": return losses
        if reduction=="avg": return losses.mean()
        

In [381]:
num_labels = 5
blank_index = num_labels-1 # last output = blank
batch_size = 1
pad = -1
T = torch.LongTensor([20])
U = torch.LongTensor([8])
y = torch.randint(low=0,high=num_labels-1,size=(U[0],)).unsqueeze(0).long()
print(y)

net_out = torch.randn(max(T), batch_size, num_labels).log_softmax(2).detach().requires_grad_()

tensor([[2, 3, 0, 2, 0, 1, 0, 2]])


In [382]:
transducer_loss = TransducerLoss()
loss_ = transducer_loss(encoder_out=encoder_out,
                        decoder_out=decoder_out,
                        targets=y,
                        input_lengths=T,
                        target_lengths=U,
                        reduction="none",
                        blank=blank_index)

print("loss:", loss)

loss.mean().backward()
print("my grad:")
# print(net_out.grad[:,1,:])
# print(net_out.grad[:,1,:].sum(1))
print(net_out.grad)
del net_out.grad

my implementation: tensor([16.4222], grad_fn=<StackBackward>)
warp-ctc: tensor([16.4222], grad_fn=<CtcLossBackward>)
my grad:
tensor([[[ 0.0447,  0.0648, -0.0627,  0.6299, -0.6767]],

        [[    nan,     nan,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,     nan,     nan]],

        [[    nan,     nan,     nan,     nan,     nan]],

        [[-0.2911,  0.2252,  0.0195,  0.1029, -0.0564]],

        [[-0.0317,  0.0706,  0.0104,  0.1224, -0.1716]],

        [[ 0.0310,  0.0926, -0.0604,  0.0115, -0.0747]],

        [[ 0.0769,  0.0672, -0.3029,  0.1345,  0.0243]],

        [[-0.0426,  0.1485, -0.2297,  0.1311, -0.0073]],

        [[-0.1356,  0.0467,  0.0261,  0.1016, -0.0388]],

        [[-0.2538,  0.0268,  0.0994,  0.1057,  0.0220]],

        [[-0.0636, -0.2955,  0.2804,  0.0576,  0.0210]],

    